In [ ]:
import numpy as np
import scipy.io as sio
import scipy.signal as signal

def downsample_eeg(data, original_rate, target_rate):
    """
    EEG 데이터를 SciPy를 사용하여 다운샘플링하는 함수
    
    매개변수:
        data (np.ndarray): 입력 데이터, 형태 (samples, channels, time samples)
        original_rate (int): 원본 샘플링 주파수 (250 Hz)
        target_rate (int): 목표 샘플링 주파수 (86 Hz)
    
    반환값:
        np.ndarray: 다운샘플링된 데이터 형태 (samples, channels, time samples)
    """
    # 원본 샘플링 주파수가 목표 샘플링 주파수보다 작으면 오류 발생
    if original_rate <= target_rate:
        raise ValueError("Original rate must be greater than target rate for downsampling.")
    
    # 데이터의 형태 (samples, channels, time samples) 가져오기
    num_samples = data.shape[2]  # time samples
    new_num_samples = int(num_samples * target_rate / original_rate)  # 새로운 time samples 계산

    # 다운샘플링된 데이터를 저장할 배열 초기화
    downsampled_data = np.zeros((data.shape[0], data.shape[1], new_num_samples))

    # 각 sample, channel, time sample에 대해 다운샘플링 수행
    for sample in range(data.shape[0]):
        for channel in range(data.shape[1]):
            # 각 채널에 대해 별도로 리샘플링 수행
            downsampled_data[sample, channel, :] = signal.resample(data[sample, channel, :], new_num_samples)

    return downsampled_data

In [ ]:
import numpy as np
from scipy import signal

def compute_psd_welch_scipy(data, sfreq, nperseg=172, fmin=0.0, fmax=None):
    """
    SciPy의 Welch 방법을 사용하여 3D 데이터(samples, channels, time samples)의 PSD(전력 스펙트럼 밀도)를 계산

    매개변수:
    - data (numpy.ndarray): 입력 데이터 (형태: (samples, channels, time samples))
    - sfreq (float): 샘플링 주파수 (86Hz)
    - nperseg (int): Welch 방법에서 각 세그먼트의 길이 (기본값: 172, 2초).
    - fmin (float): 관심 있는 최소 주파수 (기본값: 0.0)
    - fmax (float): 관심 있는 최대 주파수 (기본값: Nyquist 주파수)
    
    반환값:
    - freqs (numpy.ndarray): Array of frequency bins
    - psd_data (numpy.ndarray): PSD 값 (형태: (samples, channels, time samples))
    """
    trials, channels, time_samples = data.shape  # 데이터의 samples, channels, time samples 차원 분리
    fmax = fmax if fmax else sfreq / 2  # fmax가 None이면 Nyquist 주파수로 설정
    psd_data = []  # PSD 결과를 저장할 리스트

    for trial in range(trials):  # 각 trial에 대해 반복
        trial_data = data[trial, :, :]  # (samples, channels, time samples) 형태로 trial 데이터 선택

        # 각 trial에 대해 Welch 방법을 사용해 PSD 계산
        psd, freqs = signal.welch(trial_data, sfreq, nperseg=nperseg, noverlap=nperseg//2, 
                                   nfft=2**int(np.ceil(np.log2(nperseg))), 
                                   axis=-1, return_onesided=True)

        psd_data.append(psd)  # 현재 trial의 PSD 결과를 리스트에 추가

    psd_data = np.array(psd_data)  # (samples, channels, time samples) 형태로 결과 배열 생성
    return freqs, psd_data


In [ ]:
import numpy as np
import mne

def compute_psd_welch_mne_filter_bank(data, sfreq, nperseg=172, fmin=0.0, fmax=None):
    """
    MNE의 psd_welch를 사용하여 3D 데이터 (trials, channels, time_samples)에 대해 filterbank(4-36 Hz, 2 Hz 간격)를 사용해 PSD 계산
    
    매개변수:
    - data (numpy.ndarray): 입력 데이터, 형태는 (trials, channels, time_samples)
    - sfreq (float): 샘플링 주파수
    - nperseg (int): Welch 방법에서 사용할 세그먼트의 길이 (기본값은 172로 설정, 2초)
    - fmin (float): 관심 있는 최소 주파수 (기본값은 0.0)
    - fmax (float): 관심 있는 최대 주파수 (기본값은 Nyquist 주파수)
    
    반환값:
    - freqs (numpy.ndarray): frequency bin 배열
    - psd_data (numpy.ndarray): 형태는 (trials, channels, frequencies, bands)인 PSD 값
    """
    trials, channels, time_samples = data.shape
    fmax = fmax if fmax else sfreq / 2 
    psd_data = []

    # 주파수 대역 생성 (4-6 Hz, 6-8 Hz, ..., 34-36 Hz)
    bands = [(i, i + 2) for i in range(4, 36, 2)]
    
    for trial in range(trials):
        trial_psd = []

        for band in bands:
            lowcut, highcut = band
            
            # Welch 방법을 사용하여 PSD 계산
            psd, freqs = mne.time_frequency.psd_welch(
                data[trial:trial+1],  # 현재 trial에 대해 데이터를 slicing
                sfreq=sfreq,
                fmin=fmin, 
                fmax=fmax, 
                n_per_seg=nperseg
            )
            trial_psd.append(psd[0])  # 단일 trial에 대한 PSD 추출
        
        psd_data.append(np.array(trial_psd))

    psd_data = np.array(psd_data)  # (trials, channels, frequencies, bands) 형태로 변환
    return freqs, psd_data
